F#
======================
F# from (somewhat) first principles, immediately to practical data science examples.

F# is composed of **expressions**, which are separated by `;`s

In [27]:
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"

Installed Packages Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.2.1

In [28]:
open Plotly.NET

Chart.Point(xy = [ 1, 2; 3, 4 ])

<!-- Plotly chart will be drawn inside this DIV -->

In [51]:
[|5. .. -1. .. -5.|]

[ 5, 4, 3, 2, 1, 0, -1, -2, -3, -4, -5 ]

In [52]:
open System
open Plotly.NET

//---------------------- Generate linearly spaced vector ----------------------
let linspace (min, max, n) =
    if n <= 2 then
        failwithf "n needs to be larger then 2"

    let bw = float (max - min) / (float n - 1.)
    Array.init n (fun i -> min + (bw * float i))

//---------------------- Create example data ----------------------
let size = 100
let x = linspace (-2. * Math.PI, 2. * Math.PI, size)
let y = linspace (-2. * Math.PI, 2. * Math.PI, size)

let f x y = -(5. * x / (x ** 2. + y ** 2. + 1.))

let z = Array.init size (fun i -> Array.init size (fun j -> f x.[j] y.[i]))

Chart.Surface(zData = Array.init 100 _.[|5. .. -1. .. -5.|])

<!-- Plotly chart will be drawn inside this DIV -->

In [31]:
1; 2; 3;

3

or more commonly: lines

In [32]:
1
2
3

3

In [33]:
// this is how to write an F# comment
777 //the remainder of the line will be ignored

777

F# provides a terse syntax for representing built in data types

In [34]:
1           // this is an integer!
2.          // this is a decimal!
3, 4        // this is a tuple.     Commas `,` separate tuple elements, not collection elements
"abc"       // this is a string!
[1; 2; 3]   // this is a list!      Semicolons `;` separate collection elements
[|4; 5; 6|] // this is an array!

[ 4, 5, 6 ]

Expressions can be composed with other expressions to allow you to make complex calculations, using an expression called a **function**.

Here we take a function expression which *evaluates* its inner body (`x + 2`) with `x` set to `3`, producing the final expression `5`.

In [35]:
(fun x -> x + 2) 3

5

Functions work a lot like pipes, in both the real-world and computer IT sense of the word. They take one value as input, then evaluate their body with respect to that input, producing a single output. Because of this, they are often written with the applied argument written *before* the function using the *forward pipe operator* (`|>`).

In [36]:
3 |> (fun x -> x + 2)

5

Functions take more than parameter using *composition* (often referred to in this context as *closure*+).

If our goal is to add two numbers, let's replace `2` in our above function with `y`. But in order to do that, we need to define `y`, so let's wrap our function in another function that defines `y`.

In [37]:
(fun y -> (fun x -> x + y))

Now, we can bind `4` to `x` by piping it into our inner function:

In [38]:
(fun y -> 4 |> (fun x -> x + y))

The equivalent of what we have evaluated here is a function defined as follows:
```fsharp
(fun y -> 4 + y)
```

Now if we pipe `3` to *that* function, we finally get an `int` value:

In [39]:
3 |> (fun y -> 4 |> (fun x -> x + y))

7

It can be a bit hard to follow all the parentheses, so we can rewrite it using line breaks and spaces:

In [40]:
3 |> (fun y -> 
    4 |> (fun x ->
        x + y
))

7

What we've built here is a naive representation of a powerful expression called a `let` binding, which lets us drop the ceremony of using `fun`. A `let` binding binds a name to an expression in an enclosing scope:

In [41]:
let y = 3 in
    let x = 4 in
        x + y

7

Spaces and `in` are used here to illustrate the point that `let` bindings bind a value to a name in subsequent expressions, but in real F# code, they are rarely used (as above). We can remove the `in`s and the spaces and get something that looks a lot more like a variable definition.

In [42]:
let y = 3
let x = 4
x + y

7

It's important to remember that these aren't *variables*, but bindings.

The `=` isn't a reference operator in F#, but a function which checks equality. The following code is composed of one expression: the boolean representation of whether `x` equals `y`.

In [43]:
x = y

False

Let's revisit our composition example from earlier:

In [44]:
3 |> (fun y -> 
    4 |> (fun x ->
        x + y
))

7

The *signature* of a function tells how many and what types of parameters it takes.

We can bind our function to a name using a `let` binding, then hover our mouse over the name to view information about the signature (in VS Code).

In [45]:
3 |> (fun y -> 
    let inner = (fun x -> x + y)
    4 |> inner
)

7

The inner function takes an `int` (`x`) and produces an `int`. This is written as `int -> int` in F#.

Let's remove the `let` binding, remove the parameters, remove the line breaks for clarity, and then bind that function to a name using `let` so we can see its signature:

In [46]:
let add = (fun y -> (fun x -> x + y))

The signature is `int -> int -> int`. Similar to first signature example, we can read this as "a function that takes two `int` parameters and returns an `int`".

...However, we can see from the function body that it's actually a function that takes a single `int`, then returns a function that takes an `int` and returns an `int`.

Unlike other languages, in F#, *these signatures mean the same thing*. We can rewrite our function to not require an inner `fun`:

In [47]:
let add = (fun y x -> x + y)

In [48]:
// Now, we can write
add 2 3
// as expected

5

In [49]:
// However, similar to our "function wrapping a function" example, we can also write
let add2To = add 2
add2To 3

5

This feature is called *partial application*, and it's an extremely useful feature when used in combination with passing functions to other functions.